In [ ]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [56]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1, prev_v)

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

In [57]:
n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

x = 12
d = 0
distances = sc.parallelize([(x, d)]).partitionBy(n)
graph_history = []
while True:
  candidates = distances.join(forward_edges, n).map(step)
  graph_history = graph_history + list(candidates.toLocalIterator())
  new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist()
  count = new_distances.filter(lambda i: i[0] == 34).count()
  if count == 0:
    d += 1
    distances = new_distances.distinct()
  else:
    d += 1
    break

In [58]:
graph = graph_history
graph = list(set(graph_history))

In [59]:
graph = sorted(graph, key=lambda tup: tup[1], reverse=True)

In [60]:
current_node = 34
current_dist = 8
shortest_path = []

while current_node != 12:
    for node in graph:
        if node[0] == current_node and node[1] == current_dist:
            shortest_path.append(str(node[0]))
            current_node = node[2]
            current_dist -= 1
            break
shortest_path.append("12")
print(",".join(shortest_path[::-1]))

12,422,53,52,107,20,23,274,34
